In [2]:
## Data processing of the food_source_data

import pandas as pd

def clean_main_data(filename):
    '''
    
    '''
    (df_2013, df_2016, df_2019)= create_year_df(filename)

    final_df13 = merge_df(df_2013, 2013)
    final_df15 = merge_df(df_2016, 2016)
    final_df19 = merge_df(df_2019, 2019)

    final_df_lst = [final_df13, final_df15, final_df19]
    final_food_df = pd.concat(final_df_lst)
    
    return final_food_df

In [3]:
def create_year_df(filename):
    '''
    
    '''
    chi_food = pd.read_csv(filename)
    chi_food = chi_food.dropna()
    chi_food['license_'] = chi_food['license_'].fillna(0.0)
    chi_food['facility_type'] = chi_food['facility_type'].str.lower()
    chi_food['facility_type'] = chi_food['facility_type'].str.replace(r'[^\w\s]+', ' ', regex=True)
    chi_food['inspection_date'] = pd.to_datetime(chi_food['inspection_date'], format='%Y-%m-%d')

    s12_13 = chi_food.loc[(chi_food['inspection_date'] >= '2012-01-01') & (chi_food['inspection_date'] < '2013-12-31')]
    s15_16 = chi_food.loc[(chi_food['inspection_date'] >= '2015-01-01') & (chi_food['inspection_date'] < '2016-12-31')]
    s18_19 = chi_food.loc[(chi_food['inspection_date'] >= '2018-01-01') & (chi_food['inspection_date'] < '2019-12-31')]


    s12_13 = s12_13.drop_duplicates(['dba_name'], keep='first')
    df12_13 = s12_13.drop_duplicates(["license_"], keep='first')

    s15_16 = s15_16.drop_duplicates(['dba_name'], keep='first')
    df15_16= s15_16.drop_duplicates(["license_"], keep='first')

    s18_19 = s18_19.drop_duplicates(['dba_name'], keep='first')
    df18_19 = s18_19.drop_duplicates(["license_"], keep='first') 

    return (df12_13, df15_16, df18_19)

In [2]:
import pandas as pd

def create_year_df(FOOD_FILE):
    '''
    
    '''
    chi_food = pd.read_csv(FOOD_FILE)
    chi_food = chi_food.dropna()
    chi_food['license_'] = chi_food['license_'].fillna(0.0)
    chi_food['dba_name'] = chi_food['dba_name'].str.lower()
    chi_food['dba_name'] = chi_food['dba_name'].str.replace(r'[^\w\s]+', ' ', regex=True)
    chi_food['facility_type'] = chi_food['facility_type'].str.lower()
    chi_food['facility_type'] = chi_food['facility_type'].str.replace(r'[^\w\s]+', ' ', regex=True)
    chi_food['inspection_date'] = pd.to_datetime(chi_food['inspection_date'], format='%Y-%m-%d')

    s12_13 = chi_food.loc[(chi_food['inspection_date'] >= '2012-01-01') & (chi_food['inspection_date'] < '2013-12-31')]
    s15_16 = chi_food.loc[(chi_food['inspection_date'] >= '2015-01-01') & (chi_food['inspection_date'] < '2016-12-31')]
    s18_19 = chi_food.loc[(chi_food['inspection_date'] >= '2018-01-01') & (chi_food['inspection_date'] < '2019-12-31')]

    s12_13 = s12_13.drop_duplicates(['dba_name'], keep='first')
    zero_license_13 = s12_13.loc[s12_13['license_'] == 0.0]
    zero_df_13 = zero_license_13.iloc[1: ]
    df12_13 = s12_13.drop_duplicates(["license_"], keep='first')
    df12_13_fin = pd.concat([zero_df_13, df12_13])
    
    s15_16 = s15_16.drop_duplicates(['dba_name'], keep='first')
    zero_license_16 = s15_16.loc[s15_16['license_'] == 0.0]
    zero_df_16 = zero_license_16.iloc[1: ]
    df15_16= s15_16.drop_duplicates(["license_"], keep='first')
    df15_16_fin = pd.concat([zero_df_16, df15_16])
    
    s18_19 = s18_19.drop_duplicates(['dba_name'], keep='first')
    zero_license_19 = s18_19.loc[s18_19['license_'] == 0.0]
    zero_df_19 = zero_license_19.iloc[1: ]
    df18_19 = s18_19.drop_duplicates(["license_"], keep='first') 
    df18_19_fin = pd.concat([zero_df_19, df18_19])

    return (df12_13_fin, df15_16_fin, df18_19_fin)

In [15]:
def create_facility_dfs(df, year):
    '''
    Input df
    Returns 1 df for the given year
    '''

    df = df.reindex(columns = df.columns.tolist() + ['year'], fill_value = year)

    df_liquor = df[df['facility_type'].str.contains('liq') | df['dba_name'].str.contains('beverage')]
    df_liquor_final = df_liquor.reindex(columns = df_liquor.columns.tolist() + ['category'], \
        fill_value = 'Liquor Store') 
    
    df_convenience = df[df['facility_type'].str.contains('conv | gas') | \
        df['dba_name'].str.contains('gas sta')]
    df_convenience_final = df_convenience.reindex(columns = df_convenience.columns.tolist() \
        + ['category'], fill_value = 'Convenience Store/Gas Station')

    df_grocery = df[df['facility_type'].str.contains('groc')]
    df_grocery_final = df_grocery.reindex(columns = df_grocery.columns.tolist() + ['category'], \
        fill_value = 'Grocery Store') 

    df_restaurant = df[df['facility_type'].str.contains('rest')]
    df_restaurant_final = df_restaurant.reindex(columns = df_restaurant.columns.tolist() \
        + ['category'], fill_value = 'Restaurant')

    df_years_lst = [df_liquor_final, df_convenience_final, df_grocery_final, df_restaurant_final]
    
    return df_years_lst

In [24]:
def merge_df(df, year):
    '''
    
    '''
    df_years_lst = create_facility_dfs(df, year)
    final_duplicates = pd.concat(df_years_lst)
    final = final_duplicates.drop_duplicates(['dba_name'], keep='first')
    zero_license_final = final.loc[final['license_'] == 0.0]
    zero_df_final = zero_license_final.iloc[1: ]
    final_df = final.drop_duplicates(["license_"], keep='first') 
    final_master_df = pd.concat([zero_df_final, final_df])

    return final_master_df 

In [28]:
(a, b, c) = create_year_df('food_source_data_set.csv')
(liq, con, groc, rest) = create_facility_dfs(a, 2013)
df_13 = merge_df(a, 2013)
#df_13
test = df_13[df_13['dba_name'].str.contains('gas sta')]
test
#groc
#rest = chi_food.loc[chi_food['facility_type'] == 'restaurant']


,inspection_id,dba_name,license_,facility_type,inspection_date,latitude,longitude,year,category
3382,1319696,citgo gas station,2036895.0,grocery store,2013-08-12,41.801908,-87.621561,2013,Convenience Store/Gas Station
156349,1228348,clark gas station 1,2009255.0,gas station,2013-12-04,41.845545,-87.638419,2013,Convenience Store/Gas Station
161347,1360877,bjc gas stations inc,2283907.0,grocery store,2013-09-12,41.924379,-87.726452,2013,Convenience Store/Gas Station
163908,1192211,mobile gas station,1649830.0,grocery store,2013-09-13,41.972934,-87.777787,2013,Convenience Store/Gas Station
169123,1146443,55th gas station,2104499.0,grocery store,2013-06-25,41.793006,-87.733026,2013,Convenience Store/Gas Station
172932,1321341,mobil gas station,1980885.0,grocery store,2013-01-15,41.779849,-87.632137,2013,Convenience Store/Gas Station


In [ ]:
import pandas as pd

# Function clean_data
chi_food = pd.read_csv('food_source_data_set.csv')
chi_food = chi_food.dropna()
chi_food['license_'] = chi_food['license_'].fillna(0.0)
chi_food['facility_type'] = chi_food['facility_type'].str.lower()
chi_food['facility_type'] = chi_food['facility_type'].str.replace(r'[^\w\s]+', ' ', regex=True)
chi_food['inspection_date'] = pd.to_datetime(chi_food['inspection_date'], format='%Y-%m-%d')

s12_13 = chi_food.loc[(chi_food['inspection_date'] >= '2012-01-01') & (chi_food['inspection_date'] < '2013-12-31')]
s15_16 = chi_food.loc[(chi_food['inspection_date'] >= '2015-01-01') & (chi_food['inspection_date'] < '2016-12-31')]
s18_19 = chi_food.loc[(chi_food['inspection_date'] >= '2018-01-01') & (chi_food['inspection_date'] < '2019-12-31')]


s12_13 = s12_13.drop_duplicates(['dba_name'], keep='first')
df12_13 = s12_13.drop_duplicates(["license_"], keep='first')

s15_16 = s15_16.drop_duplicates(['dba_name'], keep='first')
df15_16= s15_16.drop_duplicates(["license_"], keep='first')

s18_19 = s18_19.drop_duplicates(['dba_name'], keep='first')
dfs18_19 = s18_19.drop_duplicates(["license_"], keep='first') 

#return (df12_13, df15_16, df18_19)

##Function create year (df, year)

df13 = df12_13.reindex(columns = df12_13.columns.tolist() + ['year'], fill_value = 2013)

df13_restaurant = df13[df13['facility_type'].str.contains('rest')]
df13_restaurant_final = df13_restaurant.reindex(columns = df13_restaurant.columns.tolist() + ['category'], fill_value = 'Restaurant')

df13_convenience = df13[df13['facility_type'].str.contains('conv')]
df13_convenience_final = df13_convenience.reindex(columns = df13_convenience.columns.tolist() + ['category'], fill_value = 'Convenience Store')

df13_grocery = df13[df13['facility_type'].str.contains('groc')]
df13_grocery_final = df13_grocery.reindex(columns = df13_grocery.columns.tolist() + ['category'], fill_value = 'Grocery Store')



#Check resturant and grocery df to see if there are duplicates

#Combine into 1 df
##returns dfyr_final

#Function create master df(all dfs)
frames = [df13_restaurant_final, df13_convenience_final, df13_grocery_final]
  
result = pd.concat(frames)


final = result.drop_duplicates(['dba_name'], keep='first')
final_final = final.drop_duplicates(["license_"], keep='first') 
print(final_final)
#returns master_df

#  df13_years_lst = create_facility_dfs(df_2013, 2013)
df16_years_lst = create_facility_dfs(df_2016, 2016)
df19_years_lst = create_facility_dfs(df_2019, 2019)


In [44]:
#function for creating category lists

#df[df['genre'].str.contains('IC')]

#filter into lists: restaurant, grocery store, convenience

#rest = chi_food.loc[chi_food['facility_type'] == 'restaurant']
def create_facility_list(df):
    '''
    Input df
    Returns lists of food facility categories
    '''
    #resturant_lst = []
    #convience_lst = []
    #grocery_lst = []

    #for i, value in df.itertuples():
    r = df[df['facility_type'].str.contains('rest')]
        #resturant_lst.append(r)
    c = df[df['facility_type'].str.contains('conv')]
        #convience_lst.append(c)
    g = df[df['facility_type'].str.contains('groc')]
        #grocery_lst.append(g)
    print(len(g))
    return (r, c, g)

        

In [ ]:
chi_food = chi_food.dropna()
    chi_food['license_'] = chi_food['license_'].fillna(0.0)
    chi_food['facility_type'] = chi_food['facility_type'].str.lower()
    chi_food['facility_type'] = chi_food['facility_type'].str.replace(r'[^\w\s]+', ' ', regex=True)
    chi_food['inspection_date'] = pd.to_datetime(chi_food['inspection_date'], format='%Y-%m-%d')

    s12_13 = chi_food.loc[(chi_food['inspection_date'] >= '2012-01-01') & (chi_food['inspection_date'] < '2013-12-31')]
    s15_16 = chi_food.loc[(chi_food['inspection_date'] >= '2015-01-01') & (chi_food['inspection_date'] < '2016-12-31')]
    s18_19 = chi_food.loc[(chi_food['inspection_date'] >= '2018-01-01') & (chi_food['inspection_date'] < '2019-12-31')]

    s12_13 = s12_13.drop_duplicates(['dba_name'], keep='first')
    zero_license_13 = s12_13.loc[s12_13['license_'] == 0.0]
    zero_df_13 = zero_license_13.iloc[1: ]
    df12_13 = s12_13.drop_duplicates(["license_"], keep='first')
    pd.concat(df_years_lst)
